<h1 style="color:green">Conway Pipeline Runner - Branch lifecycle</h1>
<p style="color:green">This notebook contains commands to manage the conway repos. In particular:</p>
    <li style="color:green"> Cross-project consistency - all are committed</li>
    <li style="color:green"> Local-remote consistency - on same commit</li>
    <li style="color:green"> Production branch isolation - uses the right tag/commit</li>
        
<p>We use <a href="https://gitpython.readthedocs.io/en/stable/tutorial.html">GitPython commands</a></p>

<p style="color:red">NB: this notebook is based on CCL's SDLC notebooks, but it is meant for on-site consultants</p>

In [1]:
import os                                                               as _os
import sys
sys.path.append(_os.path.dirname(_os.getcwd())) # So we can import tvm_notebook_utils
import chassis_nb_utils
NBU                       = chassis_nb_utils.Chassis_NB_Utils()
DFU                       = NBU.DFU
T0                        = NBU.time.perf_counter()

CONWAY installation:                conway_fork    
Jupyter using repo[branch]:      conway.ops[afin-dev]    
Installation path:               /home/alex/consultant1@CCL/dev/conway_fork    
Application:                     <class 'chassis_nb_application.Chassis_NB_Application'>    


In [2]:
USER            = "alejandro-fin"
GH_ORGANIZATION = "alejandro-fin"

DEV_PROJECT_ROOT        = "/home/alex/consultant1@CCL/dev/conway_fork"
OPERATE_PROJECT_ROOT    = "/home/alex/consultant1@CCL/operate/conway_fork"

REMOTE_ROOT     = f"https://{USER}@github.com/{GH_ORGANIZATION}"

In [3]:
CRB             = NBU.Chassis_RepoBundle()
[r.name for r in CRB.bundled_repos()]

['conway.svc',
 'conway.docs',
 'conway.test',
 'conway.scenarios',
 'conway.ops',
 'conway.acceptance',
 'conway.acceptance.docs']

In [4]:
def FLUSH():
    NBU.Application.app().logger.flush()

<h1 style="color:blue">Developer workflow</h1>
<p style="color:blue">This is to do feature work:</p>
<li style="color:blue">Choose feature branch</li>
<li style="color:blue">See status at any time</li>
<li style="color:blue">Commit to branch at any time</li>
<li style="color:blue">Deliver to integration when feature is complete</li>

In [5]:
CONWAY_LOCAL_REPOS        = ["conway.svc", "conway.acceptance", "conway.ops", "conway.test", "conway.scenarios"]
#CONWAY_LOCAL_REPOS        = ["conway.svc", "conway.ops"]
PROJECT_LOCAL_BUNDLE      = NBU.RepoBundleFactory.inferFromRepoList(CONWAY_LOCAL_REPOS)

GH_SECRETS_PATH                 = NBU.Application.app().config.config_dict['secrets']['vault_location']

dev_admin       = NBU.BranchLifecycleManager(local_root=DEV_PROJECT_ROOT, remote_root=REMOTE_ROOT, repo_bundle=PROJECT_LOCAL_BUNDLE,
                                            remote_gh_user=USER, remote_gh_organization = GH_ORGANIZATION,
                                             gh_secrets_path=GH_SECRETS_PATH)

release_admin              = NBU.BranchLifecycleManager(
                                            local_root              = OPERATE_PROJECT_ROOT, 
                                            remote_root             = REMOTE_ROOT, 
                                            repo_bundle             = PROJECT_LOCAL_BUNDLE,
                                            remote_gh_user          = USER, 
                                            remote_gh_organization  = GH_ORGANIZATION,
                                            gh_secrets_path         = GH_SECRETS_PATH)

operate_admin              = NBU.BranchLifecycleManager(
                                            local_root              = OPERATE_PROJECT_ROOT, 
                                            remote_root             = REMOTE_ROOT, 
                                            repo_bundle             = PROJECT_LOCAL_BUNDLE,
                                            remote_gh_user          = USER, 
                                            remote_gh_organization  = GH_ORGANIZATION,
                                            gh_secrets_path         = GH_SECRETS_PATH)


In [6]:
for ri in PROJECT_LOCAL_BUNDLE.bundled_repos():
    print(ri.name)

conway.svc
conway.acceptance
conway.ops
conway.test
conway.scenarios


<h3 style="color:blue">Choose what to work on</h3>

In [7]:
FEATURE_BRANCH  = "afin-dev"
#FEATURE_BRANCH  = "task-conway-2"

In [8]:
with NBU.Profiler("Work on feature"):
    await dev_admin.work_on_feature(FEATURE_BRANCH)
FLUSH()



[12.219 sec Task-6 - MainThread@repo_administration:363]	
----------- conway.ops (local) -----------
[12.455 sec Task-6 - MainThread@repo_administration:363]	@ 'afin-dev' (local):

Your branch is up to date with 'origin/afin-dev'.
[12.385 sec Task-8 - MainThread@repo_administration:363]	
----------- conway.scenarios (local) -----------
[12.504 sec Task-8 - MainThread@repo_administration:363]	@ 'afin-dev' (local):

Your branch is up to date with 'origin/afin-dev'.
[12.396 sec Task-9 - MainThread@repo_administration:363]	
----------- conway.test (local) -----------
[12.514 sec Task-9 - MainThread@repo_administration:363]	@ 'afin-dev' (local):

Your branch is up to date with 'origin/afin-dev'.
[12.414 sec Task-7 - MainThread@repo_administration:363]	
----------- conway.svc (local) -----------
[12.489 sec Task-7 - MainThread@repo_administration:363]	@ 'afin-dev' (local):

Your branch is up to date with 'origin/afin-dev'.
[12.427 sec Task-10 - MainThread@repo_administration:363]	
--------

<h3 style="color:blue">Refresh from integration</h3>
<p style="color:blue">This ensures we will work on top of what others have previously delivered</p>

In [9]:
with NBU.Profiler("Refresh from integration"):
    await dev_admin.refresh_from_integration(FEATURE_BRANCH)
FLUSH()



[20.501 sec Task-12 - MainThread@repo_administration:363]	
----------- conway.svc (local) -----------
[20.524 sec Task-12 - MainThread@filesystem_repo_inspector:357]	local = '/home/alex/consultant1@CCL/dev/conway_fork/conway.svc'
[20.777 sec Task-12 - MainThread@filesystem_repo_inspector:365]	@ 'integration' (local):

Your branch is up to date with 'origin/integration'.
[22.879 sec Task-12 - MainThread@filesystem_repo_inspector:368]	'integration' (remote) -> 'integration' (local):

Already up to date.
[22.900 sec Task-12 - MainThread@filesystem_repo_inspector:373]	@ 'afin-dev' (local):

Your branch is up to date with 'origin/afin-dev'.
[22.921 sec Task-12 - MainThread@filesystem_repo_inspector:342]	'integration' (local) -> 'afin-dev' (local):

Already up to date.
[20.550 sec Task-13 - MainThread@repo_administration:363]	
----------- conway.ops (local) -----------
[20.579 sec Task-13 - MainThread@filesystem_repo_inspector:357]	local = '/home/alex/consultant1@CCL/dev/conway_fork/conway

<h3 style="color:blue">Status</h3>

In [10]:
with NBU.Profiler("Gathering repo information"):
    stats_df    = await dev_admin.repo_stats() 
FLUSH()
stats_df



[30.602 sec Task-17 - MainThread@<source location undetermined>]	Gathering repo information completed in 0.75 sec


,Repo,Local/Remote,Current Branch,# Untracked files,# Modified files,# Deleted files,Last commit,Last commit timestamp,Last commit hash
0,conway.acceptance,Local,afin-dev,0,0,0,Standardized on Logger.log instead of Applicat...,241102.111518,1df8636848c7a4f5a9c2b2064ea6fda468cbc602
5,conway.acceptance,Remote,master,0,0,0,[PR #43] Merge integration -> master (remote),241106.152128,038037e8a14482fa48499b11c3a47372a246445c
3,conway.ops,Local,afin-dev,0,0,0,[afin-dev] Instrumented ScheduledBasedLogSorter,241106.141303,dbee5d4b71175df847f815252791fd34e9277872
7,conway.ops,Remote,master,0,0,0,[PR #69] Merge integration -> master (remote),241106.152128,dafa7bd367c97246ff26332d576cfcbe94c94c14
2,conway.scenarios,Local,afin-dev,0,0,0,[PR #45] Merge master -> integration (remote),240531.161721,95564eee1923878fb9f8b5f02ac9eaac048b7d90
6,conway.scenarios,Remote,master,0,0,0,[PR #46] Merge integration -> master (remote),241106.152128,cfea3235b38f661e5366a28af827c05b9aa01e78
4,conway.svc,Local,afin-dev,0,0,0,[afin-dev] Instrumented ScheduledBasedLogSorter,241106.141303,87394f0667ee49c17a1b67d990f5d6b89086e72e
9,conway.svc,Remote,master,0,0,0,[PR #70] Merge integration -> master (remote),241106.152128,6a594617e388a73bc8965ecc98d2bdcf8647856f
1,conway.test,Local,afin-dev,0,0,0,[afin-dev] Instrumented ScheduledBasedLogSorter,241106.141303,01ef86921083b99ea5b9082b2d43987b997494e4
8,conway.test,Remote,master,0,0,0,[PR #46] Merge integration -> master (remote),241106.152128,1cc050f773acf3c33e565fce4e0879fd9ba1a549


<h3 style="color:blue">Commit and push</h3>
<p style="color:blue">Push is to remote feature branch, so it is done for backup purposes</p>

In [12]:
COMMIT_MSG                         = "[" + FEATURE_BRANCH + "] " + ""

COMMIT_MSG

'[afin-dev] Instrumented ScheduledBasedLogSorter'

In [14]:
with NBU.Profiler("Committing work in '" + FEATURE_BRANCH + "'"):
    await dev_admin.commit_feature(FEATURE_BRANCH, COMMIT_MSG)
FLUSH()



[264.766 sec Task-55 - MainThread@repo_administration:363]	
----------- conway.svc (local) -----------
[264.790 sec Task-55 - MainThread@repo_administration:363]	local = '/home/alex/consultant1@CCL/dev/conway_fork/conway.svc'
[264.904 sec Task-55 - MainThread@repo_administration:363]	@ 'afin-dev' (local):

On branch afin-dev
Your branch is up to date with 'origin/afin-dev'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   src/conway/async_utils/schedule_based_log_sorter.py
	modified:   src/conway/observability/logger.py

no changes added to commit (use "git add" and/or "git commit -a")
[264.989 sec Task-55 - MainThread@repo_administration:363]	'afin-dev' (working tree) -> 'afin-dev' (staging area):

[265.048 sec Task-55 - MainThread@repo_administration:363]	'afin-dev' (staging area) -> 'afin-dev' (local):
[afin-dev 87394f0] [afin-dev] Instrumented Scheduled

<h3 style="color:blue">Deliver to integration</h3>

In [19]:
# This should only be done after all tests are passing in the feature branch
#
with NBU.Profiler("Delivering to integration"):
    await dev_admin.complete_feature(FEATURE_BRANCH)
FLUSH()



[728.225 sec Task-100 - MainThread@repo_administration:363]	
----------- conway.scenarios (local) -----------
[728.252 sec Task-100 - MainThread@repo_administration:363]	local = '/home/alex/consultant1@CCL/dev/conway_fork/conway.scenarios'
[728.408 sec Task-100 - MainThread@repo_administration:363]	@ 'afin-dev' (local):

On branch afin-dev
Your branch is up to date with 'origin/afin-dev'.

nothing to commit, working tree clean
[728.466 sec Task-100 - MainThread@repo_administration:363]	@ 'integration' (local):

Your branch is up to date with 'origin/integration'.
[730.573 sec Task-100 - MainThread@repo_administration:363]	integration (remote) ->'integration (local)':

Already up to date.
[730.721 sec Task-100 - MainThread@repo_administration:363]	@ 'afin-dev' (local):

Your branch is up to date with 'origin/afin-dev'.
[730.799 sec Task-100 - MainThread@repo_administration:363]	'integration' (local) - afin-dev' (local):

Already up to date.
[730.940 sec Task-100 - MainThread@repo_admi

<h1 style="color:green">Release workflow</h1>
<p style="color:green">Publishes a release:</p>
<li style="color:green">PR from integration to master</li>
<li style="color:green">Update operate branch from master</li>

<h3 style="color:green">PR from integration to master</h3>

In [9]:
# This should only be done after the production-like simulation passes successfully
#if True:
#    release_admin.pull_request_integration_to_master()
if True:
    with NBU.Profiler("PR from integration to master"):
        await release_admin.pull_request_integration_to_master()
FLUSH()

DEBUG: In GitHub_RepoInspector: parent_url='https://alejandro-fin@github.com/alejandro-fin'  repo_name='conway.acceptance'
DEBUG: In GitHub_RepoInspector: parent_url='https://alejandro-fin@github.com/alejandro-fin'  repo_name='conway.ops'
DEBUG: In GitHub_RepoInspector: parent_url='https://alejandro-fin@github.com/alejandro-fin'  repo_name='conway.scenarios'
DEBUG: In GitHub_RepoInspector: parent_url='https://alejandro-fin@github.com/alejandro-fin'  repo_name='conway.svc'
DEBUG: In GitHub_RepoInspector: parent_url='https://alejandro-fin@github.com/alejandro-fin'  repo_name='conway.test'


[25.474 sec Task-11 - MainThread@repo_administration:363]	
----------- conway.acceptance (remote) -----------
[25.497 sec Task-11 - MainThread@repo_administration:363]	
----------- conway.ops (remote) -----------
[25.536 sec Task-11 - MainThread@repo_administration:363]	
----------- conway.scenarios (remote) -----------
[25.573 sec Task-11 - MainThread@repo_administration:363]	
----------- conway.svc 

In [10]:
# NB: due to the asynchronous nature, you may need to do this twice to make sure all changes cascade through.
#    This is probably a bug to fix: we shouldn't try to refresh the local from the remote operate branch until the remote operate
#    branch is updated from master and the various pull requests are done.
if True:
    with NBU.Profiler("Publish release"):
        await release_admin.publish_release()
FLUSH()



[351.435 sec Task-27 - MainThread@repo_administration:363]	
----------- conway.acceptance (remote) -----------
[351.452 sec Task-27 - MainThread@repo_administration:363]	
----------- conway.acceptance (local) -----------
[351.468 sec Task-27 - MainThread@repo_administration:363]	
----------- conway.ops (remote) -----------
[351.492 sec Task-27 - MainThread@repo_administration:363]	
----------- conway.ops (local) -----------
[351.514 sec Task-27 - MainThread@repo_administration:363]	
----------- conway.scenarios (remote) -----------
[351.535 sec Task-27 - MainThread@repo_administration:363]	
----------- conway.scenarios (local) -----------
[351.561 sec Task-27 - MainThread@repo_administration:363]	
----------- conway.svc (remote) -----------
[351.584 sec Task-27 - MainThread@repo_administration:363]	
----------- conway.svc (local) -----------
[351.600 sec Task-27 - MainThread@repo_administration:363]	
----------- conway.test (remote) -----------
[351.615 sec Task-27 - MainThread@repo_

<h1 style="color:purple">Operate workflow</h1>

<h3 style="color:purple">Status</h3>

In [ ]:
#operate_admin   = NBU.VM_BranchLifecycleManager(local_root = OPERATE_ROOT, remote_root =REMOTE_ROOT)

In [9]:
#with NBU.Profiler("Gathering operate repo information"):
#    stats_df    = operate_admin.repo_stats() 
with NBU.Profiler("Gathering operate repo information"):
    stats_df    = await operate_admin.repo_stats() 
FLUSH()
stats_df



[29.748 sec Task-11 - MainThread@<source location undetermined>]	Gathering operate repo information completed in 0.79 sec


,Repo,Local/Remote,Current Branch,# Untracked files,# Modified files,# Deleted files,Last commit,Last commit timestamp,Last commit hash
1,conway.acceptance,Local,operate,0,0,0,[PR #41] Merge master -> operate (remote),240531.160603,f59e0357d206137aa05a5851f161525ded2e2ad8
8,conway.acceptance,Remote,master,0,0,0,[PR #43] Merge integration -> master (remote),241106.152128,038037e8a14482fa48499b11c3a47372a246445c
2,conway.ops,Local,operate,0,0,0,[PR #67] Merge master -> operate (remote),240531.160603,55448c082e436603fb826b6e784465b57b50e65f
6,conway.ops,Remote,master,0,0,0,[PR #69] Merge integration -> master (remote),241106.152128,dafa7bd367c97246ff26332d576cfcbe94c94c14
0,conway.scenarios,Local,operate,0,0,0,[PR #47] Merge master -> operate (remote),241106.152353,ccfbfa5b48724093e62e9f6e87c7a09ba398068a
5,conway.scenarios,Remote,master,0,0,0,[PR #46] Merge integration -> master (remote),241106.152128,cfea3235b38f661e5366a28af827c05b9aa01e78
3,conway.svc,Local,operate,0,0,0,[PR #68] Merge master -> operate (remote),240531.160603,e69eb8df64913fe56714423ba97eb627410da362
9,conway.svc,Remote,master,0,0,0,[PR #70] Merge integration -> master (remote),241106.152128,6a594617e388a73bc8965ecc98d2bdcf8647856f
4,conway.test,Local,operate,0,0,0,[PR #47] Merge master -> operate (remote),241106.152353,76d5015d5c79aa5e59d98afec534485448855977
7,conway.test,Remote,master,0,0,0,[PR #46] Merge integration -> master (remote),241106.152128,1cc050f773acf3c33e565fce4e0879fd9ba1a549


<h3 style="color:purple">Commit and push</h3>

In [ ]:
COMMIT_MSG                         = "Hot fix in operate"
COMMIT_MSG

In [ ]:
#OPERATE_BRANCH                     = NBU.VM_RepoAdministration.OPERATE_BRANCH#\
#
#with NBU.Profiler("Committing work in '" + OPERATE_BRANCH + "'"):
#    operate_admin.commit_feature(OPERATE_BRANCH, COMMIT_MSG)

with NBU.Profiler("Committing work in operate branch"):
    await operate_admin.commit_hot_fix(COMMIT_MSG)
FLUSH()

<h3 style="color:purple">Publish a hot fix</h3>
<p style="color:purple">This flow from operate to master and from there to integration</p>

In [ ]:
#with NBU.Profiler("Publishing hot fix"):
#    operate_admin.publish_hot_fix()

with NBU.Profiler("Publishing hot fix"):
    await operate_admin.publish_hot_fix()
FLUSH()

<h1 style="color:red">Removing a branch</h1>

In [ ]:
BRANCH_TO_REMOVE = "story_xyz"

In [ ]:
if False:
    with NBU.Profiler("Removing branch"):
        dev_admin.remove_feature_branch(BRANCH_TO_REMOVE)

<h1 style="color:brown">Scratch</h1>